In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments, ViTImageProcessor,AutoImageProcessor
import os
from PIL import Image
from sklearn.metrics import accuracy_score
import numpy as np
import pyarrow.parquet as pq
import io
from sklearn.model_selection import train_test_split
from peft import LoraConfig, TaskType, get_peft_model,PeftModel, PeftConfig
#from transformers import LoRAConfig, LoRAAdapter
from utils import get_mnist_data, get_EuroSAT_data,CustomTensorDataset, get_cifar10_data, get_car_data,get_fruits_data, get_GTSRB_data, get_DTD_data, get_resis_data, get_grabage_data, get_plants_data
import random
from tqdm import tqdm

def set_random():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    # 如果使用 GPU，还需固定 GPU 相关随机数
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # 用于多 GPU 情况
    # 确保卷积操作等确定性（针对特定卷积算法）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def test_result():
    sum1 = 0
    set_random()
    for i in tqdm(range(1000)):
        with torch.no_grad():
            outputs = model(test_dataset[i]["pixel_values"].unsqueeze(0).cuda())
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
        if predicted_class != int(test_dataset[i]['labels']):
            #print(predicted_class, int(test_dataset[i]['labels']))
            sum1 += 1
    print(1-sum1/1000)
    return 1-sum1/1000
    

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}
model_dir = "./vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_dir, torch_dtype="auto")
feature_extractor = ViTImageProcessor.from_pretrained(model_dir)

# lora_config = LoraConfig(
#         #task_type=TaskType.CAUSAL_LM,
#         #task_type=TaskType.SEQ_CLS,
#         task_type=TaskType.FEATURE_EXTRACTION,
#         target_modules=[ "intermediate.dense", "output.dense"],
#         inference_mode=False,  # 训练模式
#         r=8,  # Lora 秩
#         lora_alpha=32,  # 等效于lr=lr*lora_alpha/r
#         lora_dropout=0.1
    # )

lora_config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=[ "intermediate.dense", "output.dense"],  #["query", "value"]
        lora_dropout=0.1,
        bias="none",
        modules_to_save=["classifier"],
    )

model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()


/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CustomTensorDataset(Dataset):
    def __init__(self, tensors, labels, mode= "PNG_code", transform=None): #"JPG"  Tensor
        self.tensors = tensors
        self.labels = labels
        self.transform = transform
        self.mode = mode
        assert len(self.tensors) == len(self.labels)  # "张量数量和标签数量必须相同"
    def __len__(self):
        return len(self.tensors)
    def __getitem__(self, idx):
        label = self.labels[idx]
        return {"pixel_values": self.tensors[idx], "labels": torch.tensor(label)}

In [7]:
task_functions = {"mnist": get_mnist_data,"EuroSAT": get_EuroSAT_data,"cifar10": get_cifar10_data,"car": get_car_data,"fruits": get_fruits_data,"GTSRB": get_GTSRB_data,"DTD": get_DTD_data,"resis": get_resis_data,"grabage": get_grabage_data, "plants": get_plants_data}
train_list = []
train_label = []
# test_list = []
# test_label = []
for j, task in enumerate([ "mnist", "EuroSAT", "cifar10", "car", "fruits","GTSRB", "DTD","resis", "grabage","plants"]):
    train_dataset, test_dataset = task_functions[task]()
    train_list += [train_dataset[i]['pixel_values'] for i in range(min(5000, len(train_dataset)))]
    train_label += [j] * min(5000, len(train_dataset))
    # test_list += [test_dataset[i]['pixel_values'] for i in range(min(1000, len(test_dataset)))]
    # test_label += [j] * min(1000, len(test_dataset))

In [8]:

train_dataset = CustomTensorDataset(train_list, train_label)
test_dataset = CustomTensorDataset(test_list, test_label)

In [10]:
from torchvision import datasets, transforms, models
model1 = models.resnet18(pretrained=False, num_classes=10).cuda()  # 修改分类数
model1.load_state_dict(torch.load("./router_model.pth"))

/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1536254/2176203441.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value 

<All keys matched successfully>

In [14]:
# 2. 加载训练/验证数据
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
train_dataset_ = [[i['pixel_values'],i["labels"]] for i in train_dataset]
test_dataset_ = [[i['pixel_values'],i["labels"]] for i in test_dataset]
train_loader = DataLoader(train_dataset_+test_dataset_, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset_, batch_size=32, shuffle=False)

# 3. 加载 ResNet18 模型（不使用 ImageNet 预训练）
# model = models.resnet18(pretrained=False, num_classes=10)  # 修改分类数
# model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# 4. 损失函数 + 优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 5. 训练
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_acc = 0
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        # print(images,labels)
        # print(images["pixel_values"],labels["labels"])
        # break
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")

    # 验证
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    print(f"Validation Accuracy: {acc:.2f}%")
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "./router_model.pth")
        print(f"✅ Saved new best model with accuracy {acc:.2f}%")

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Loss: 50.5148


  2%|▏         | 1/50 [01:09<57:09, 69.98s/it]

Validation Accuracy: 99.42%
✅ Saved new best model with accuracy 99.42%
Epoch [2/50], Loss: 42.6233


  4%|▍         | 2/50 [02:19<55:56, 69.93s/it]

...

Validation Accuracy: 99.94%
Epoch [49/50], Loss: 6.9718


 98%|█████████▊| 49/50 [57:05<01:09, 69.57s/it]

Validation Accuracy: 99.90%
Epoch [50/50], Loss: 10.4520


100%|██████████| 50/50 [58:15<00:00, 69.90s/it]

Validation Accuracy: 99.95%


In [ ]:
model.eval()
correct = 0
total = 0

test_dataset_ = [[i['pixel_values'],i["labels"]] for i in test_dataset]
train_loader = DataLoader(train_dataset_+test_dataset_, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset_, batch_size=32, shuffle=False)
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        for i in range(len(labels)):
            if preds[i] != labels[i]:
                print(preds[i], labels[i])
        # break
        total += labels.size(0)

acc = 100 * correct / total
print(f"Validation Accuracy: {acc:.2f}%")